<a href="https://colab.research.google.com/github/rukevweubio/ETLpipelinedesign/blob/main/ETLppelinedesign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#import the necessary libeary
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import datetime as datetime
import datetime as dt
import csv

In [3]:
df = pd.read_csv('/content/ziko_logistics_data (1) (1).csv')
df

,Transaction_ID,Date,Customer_ID,Product_ID,Quantity,Unit_Price,Total_Cost,Discount_Rate,Sales_Channel,Order_Priority,...,Return_Reason,Payment_Type,Taxable,Region,Country,Customer_Name,Customer_Phone,Customer_Email,Customer_Address,Product_List_Title
0,200,2020-01-01 20:32:25.945945945,1086,536,3,120.436821,8265.374549,0.20,Online,High,...,Wrong Item,Wire Transfer,False,West,Canada,Customer 200,+1-652-572-9306,customer.200.78@example.com,"275 Second St, Phoenix, USA",Product 53
1,321,2020-01-02 06:55:08.108108108,1078,523,6,475.724994,4047.850479,NaN,Reseller,Critical,...,Damaged,PayPal,True,South,Mexico,Customer 321,+1-311-186-5760,customer.321.13@sample.com,"478 Third St, New York, USA",Product 33
2,989,2020-01-06 08:12:58.378378378,1077,535,3,146.400556,NaN,0.05,Direct,Critical,...,Damaged,PayPal,True,West,Canada,Customer 989,+1-922-606-9032,customer.989.99@example.com,"843 Second St, Phoenix, USA",Product 6
3,682,2020-01-07 22:03:14.594594594,1027,546,6,19.373194,8194.281993,NaN,Reseller,Medium,...,Wrong Item,Cash,True,South,Canada,Customer 682,+1-237-853-5808,customer.682.66@demo.com,"153 Main St, Phoenix, USA",Product 68
4,484,2020-01-07 07:08:06.486486486,1052,556,8,193.221313,8331.329249,0.20,Direct,Low,...,Late,Cash,False,South,Mexico,Customer 484,+1-986-360-9109,customer.484.3@sample.com,"264 Second St, New York, USA",Product 89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,250,2020-01-05 02:22:42.162162162,1095,533,5,490.760429,1138.724711,NaN,Reseller,Medium,...,Damaged,Credit Card,True,South,Canada,Customer 250,+1-875-504-2878,customer.250.26@sample.com,"664 Main St, Chicago, USA",Product 16
1001,412,2020-01-07 11:27:34.054054054,1008,542,9,236.240910,1786.865600,0.10,Online,Medium,...,Not Specified,Cash,True,South,USA,Customer 412,+1-645-470-5077,customer.412.80@sample.com,"699 Third St, Houston, USA",Product 38
1002,816,2020-01-06 02:48:38.918918918,1060,511,6,NaN,6443.682500,0.20,Partner,Medium,...,Not Specified,Cash,False,West,Mexico,Customer 816,+1-162-396-1771,customer.816.55@sample.com,"453 Fourth St, Los Angeles, USA",Product 48
1003,702,2020-01-09 08:38:55.135135135,1054,504,6,NaN,4814.744796,NaN,Online,Medium,...,Damaged,Cash,True,West,Canada,Customer 702,+1-130-713-1098,customer.702.30@sample.com,"438 Fourth St, Houston, USA",Product 38


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Transaction_ID         1005 non-null   int64  
 1   Date                   1005 non-null   object 
 2   Customer_ID            1005 non-null   int64  
 3   Product_ID             1005 non-null   int64  
 4   Quantity               1005 non-null   int64  
 5   Unit_Price             904 non-null    float64
 6   Total_Cost             905 non-null    float64
 7   Discount_Rate          714 non-null    float64
 8   Sales_Channel          1005 non-null   object 
 9   Order_Priority         1005 non-null   object 
 10  Warehouse_Code         1005 non-null   object 
 11  Ship_Mode              1005 non-null   object 
 12  Delivery_Status        1005 non-null   object 
 13  Customer_Satisfaction  1005 non-null   object 
 14  Item_Returned          1005 non-null   bool   
 15  Retu

In [6]:
df.isnull().sum()

,0
Transaction_ID,0
Date,0
Customer_ID,0
Product_ID,0
Quantity,0
Unit_Price,101
Total_Cost,100
Discount_Rate,291
Sales_Channel,0
Order_Priority,0


In [7]:
missing_values=[missing for missing in df.columns if df[missing].isnull().sum()>0]
missing_values

['Unit_Price', 'Total_Cost', 'Discount_Rate', 'Return_Reason']

In [8]:
df = df.fillna({

'Unit_Price':0.0,
'Total_Cost':0.0,
'Discount_Rate':0.0,
 'Return_Reason':'unkown'})

In [9]:
df.isnull().sum()

,0
Transaction_ID,0
Date,0
Customer_ID,0
Product_ID,0
Quantity,0
Unit_Price,0
Total_Cost,0
Discount_Rate,0
Sales_Channel,0
Order_Priority,0


In [10]:
df.isnull().sum()

,0
Transaction_ID,0
Date,0
Customer_ID,0
Product_ID,0
Quantity,0
Unit_Price,0
Total_Cost,0
Discount_Rate,0
Sales_Channel,0
Order_Priority,0


In [11]:
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Date'] = pd.to_datetime(df['Date']).dt.date
df['Day'] = pd.to_datetime(df['Date']).dt.day_name()
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()
df['Year'] = pd.to_datetime(df['Date']).dt.year

In [12]:
#design each table
#customer table
customer = df[['Customer_ID','Customer_Name','Customer_Phone', 'Customer_Email', 'Customer_Address','Region', 'Country'\
               ,'Customer_Satisfaction']].drop_duplicates()
customer =customer.drop_duplicates(subset='Customer_ID')
customer

,Customer_ID,Customer_Name,Customer_Phone,Customer_Email,Customer_Address,Region,Country,Customer_Satisfaction
0,1086,Customer 200,+1-652-572-9306,customer.200.78@example.com,"275 Second St, Phoenix, USA",West,Canada,Neutral
1,1078,Customer 321,+1-311-186-5760,customer.321.13@sample.com,"478 Third St, New York, USA",South,Mexico,Satisfied
2,1077,Customer 989,+1-922-606-9032,customer.989.99@example.com,"843 Second St, Phoenix, USA",West,Canada,Unsatisfied
3,1027,Customer 682,+1-237-853-5808,customer.682.66@demo.com,"153 Main St, Phoenix, USA",South,Canada,Unsatisfied
4,1052,Customer 484,+1-986-360-9109,customer.484.3@sample.com,"264 Second St, New York, USA",South,Mexico,Satisfied
...,...,...,...,...,...,...,...,...
303,1019,Customer 617,+1-702-454-2196,customer.617.90@sample.com,"766 Fourth St, Chicago, USA",North,Canada,Satisfied
350,1066,Customer 368,+1-468-754-2445,customer.368.61@sample.com,"121 Third St, Los Angeles, USA",East,USA,Satisfied
374,1015,Customer 477,+1-925-956-9167,customer.477.91@example.com,"525 Main St, Houston, USA",East,Canada,Neutral
397,1091,Customer 499,+1-171-212-9948,customer.499.44@demo.com,"550 Main St, Phoenix, USA",North,Mexico,Unsatisfied


In [13]:
# Date table
#date table
date=df[['Date','Day', 'Month', 'Year']].drop_duplicates()
date


,Date,Day,Month,Year
0,2020-01-01,Wednesday,January,2020
1,2020-01-02,Thursday,January,2020
2,2020-01-06,Monday,January,2020
3,2020-01-07,Tuesday,January,2020
8,2020-01-08,Wednesday,January,2020
14,2020-01-05,Sunday,January,2020
24,2020-01-03,Friday,January,2020
25,2020-01-09,Thursday,January,2020
28,2020-01-04,Saturday,January,2020


In [14]:
#product table
product =df[['Product_ID','Order_Priority', 'Warehouse_Code', 'Ship_Mode', 'Delivery_Status','Item_Returned', 'Return_Reason'\
       ,'Payment_Type','Product_List_Title']].drop_duplicates()
product =product.drop_duplicates(subset='Product_ID')


product.head(10)

,Product_ID,Order_Priority,Warehouse_Code,Ship_Mode,Delivery_Status,Item_Returned,Return_Reason,Payment_Type,Product_List_Title
0,536,High,WH-3,2-Day,Cancelled,False,Wrong Item,Wire Transfer,Product 53
1,523,Critical,WH-1,Overnight,Backorder,True,Damaged,PayPal,Product 33
2,535,Critical,WH-1,Overnight,Pending,True,Damaged,PayPal,Product 6
3,546,Medium,WH-1,Express,Pending,False,Wrong Item,Cash,Product 68
4,556,Low,WH-2,2-Day,Delivered,True,Late,Cash,Product 89
5,549,Medium,WH-3,2-Day,Backorder,True,Not Specified,PayPal,Product 27
6,547,Critical,WH-3,Express,Cancelled,False,Late,Credit Card,Product 83
7,542,Critical,WH-2,Overnight,Delivered,True,Not Specified,Credit Card,Product 13
8,533,Medium,WH-1,Express,Delivered,True,Late,PayPal,Product 54
9,529,Critical,WH-3,Overnight,Pending,True,Late,Credit Card,Product 35


In [15]:
transaction =df.merge(customer, on=['Customer_ID','Customer_Name','Customer_Phone', 'Customer_Email', 'Customer_Address','Region', 'Country'\
               ,'Customer_Satisfaction'], how ='left')\
               .merge(product, on=['Product_ID','Order_Priority', 'Warehouse_Code', 'Ship_Mode', 'Delivery_Status','Item_Returned', 'Return_Reason'\
       ,'Payment_Type','Product_List_Title'], how ='left')\
       .merge(date,on=['Date','Day', 'Month', 'Year'], how ='left')

In [16]:
# transaction table
transaction = transaction[['Transaction_ID', 'Customer_ID', 'Product_ID','Date' ,'Quantity'\
       ,'Unit_Price', 'Total_Cost', 'Discount_Rate', 'Sales_Channel', 'Taxable']].drop_duplicates()
transaction

,Transaction_ID,Customer_ID,Product_ID,Date,Quantity,Unit_Price,Total_Cost,Discount_Rate,Sales_Channel,Taxable
0,200,1086,536,2020-01-01,3,120.436821,8265.374549,0.20,Online,False
1,321,1078,523,2020-01-02,6,475.724994,4047.850479,0.00,Reseller,True
2,989,1077,535,2020-01-06,3,146.400556,0.000000,0.05,Direct,True
3,682,1027,546,2020-01-07,6,19.373194,8194.281993,0.00,Reseller,True
4,484,1052,556,2020-01-07,8,193.221313,8331.329249,0.20,Direct,False
...,...,...,...,...,...,...,...,...,...,...
1000,250,1095,533,2020-01-05,5,490.760429,1138.724711,0.00,Reseller,True
1001,412,1008,542,2020-01-07,9,236.240910,1786.865600,0.10,Online,True
1002,816,1060,511,2020-01-06,6,0.000000,6443.682500,0.20,Partner,False
1003,702,1054,504,2020-01-09,6,0.000000,4814.744796,0.00,Online,True


In [ ]:
#load date into postgres database
import psycopg2

conn = None
try:
    print('Connected to the PostgreSQL database')
    connection = psycopg2.connect(
        user='postgres',
        host='localhost',
        dbname='nugabank',
        password='1',
        port=5432
    )
    cursor = connection.cursor()
    cursor.execute('SELECT version()')
    result = cursor.fetchone()
    print(f'The PostgreSQL database version: {result}')
except (Exception, psycopg2.DatabaseError) as error:
    print(f'Error: {error}')
finally:
    if conn is not None:
        cursor.close()
        connection.close()
        print('Database connection closed.')



In [18]:
def load_data_database():
    conn=psycopg2.connect(

        user='postgres',
        host='localhost',
        dbname='nugabank',
        password='1',
        port=5432
    )
    return conn


In [ ]:
import psycopg2

def load_data_database():
    return psycopg2.connect(
        user='postgres',
        host='localhost',
        dbname='nugabank',
        password='1',
        port=5432
    )

def create_table():
    conn = load_data_database()
    cur = conn.cursor()
    query = '''
    DROP TABLE IF EXISTS customer CASCADE;
    DROP TABLE IF EXISTS product CASCADE;
    DROP TABLE IF EXISTS transaction CASCADE;
    DROP TABLE IF EXISTS date CASCADE;

    CREATE TABLE IF NOT EXISTS customer(
        Customer_ID SERIAL PRIMARY KEY,
        Customer_Name VARCHAR(1000),
        Customer_Phone VARCHAR(1000),
        Customer_Email VARCHAR(1000),
        Customer_Address VARCHAR(1000),
        Region VARCHAR(1000),
        Country VARCHAR(1000),
        Customer_Satisfaction VARCHAR(1000)
    );

    CREATE TABLE IF NOT EXISTS date(
        DateID SERIAL PRIMARY KEY,
        Date DATE,
        Day VARCHAR(50),
        Month VARCHAR(60),
        Year INT
    );

    CREATE TABLE IF NOT EXISTS product(
        Product_ID SERIAL PRIMARY KEY,
        Order_Priority VARCHAR(1000),
        Warehouse_Code VARCHAR(1000),
        Ship_Mode VARCHAR(1000),
        Delivery_Status VARCHAR(1000),
        Item_Returned VARCHAR(1000),
        Return_Reason VARCHAR(1000),
        Payment_Type VARCHAR(1000),
        Product_List_Title VARCHAR(1000)
    );

    CREATE TABLE IF NOT EXISTS transaction(
        Transaction_ID SERIAL PRIMARY KEY,
        Customer_ID INT,
        Product_ID INT,
        Date DATE,
        Quantity INT,
        Unit_Price FLOAT,
        Total_Cost FLOAT,
        Discount_Rate FLOAT,
        Sales_Channel VARCHAR(1000),
        Taxable BOOL,
        FOREIGN KEY (Customer_ID) REFERENCES customer (Customer_ID),
        FOREIGN KEY (Product_ID) REFERENCES product (Product_ID)

    );
    '''
    cur.execute(query)
    conn.commit()
    cur.close()
    conn.close()

# Call the function to create the tables
create_table()
print('Tables created successfully')

In [20]:
#create a csv copy for each table
customer.to_csv('customer.csv', index=False)
transaction.to_csv('transaction.csv', index=False)
date.to_csv('date.csv', index=False)
product.to_csv('product.csv', index=False)

In [ ]:
#load the product table
def load_date_into_table(file):
    conn = load_data_database()
    cur = conn.cursor()
    with open(file, 'r') as filename:
        reader = csv.reader(filename)
        next(reader)  # Skip the header row
        for row in reader:
            cur.execute('''
                INSERT INTO customer (
                    Customer_ID, Customer_Name, Customer_Phone, Customer_Email,
                    Customer_Address, Region, Country, Customer_Satisfaction
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
            ''', row)
    conn.commit()
    cur.close()
    conn.close()

filepath = r'customer.csv'
load_date_into_table(filepath)

In [ ]:
#load date into the product table
def load_date_into_table(file):
    conn = load_data_database()
    cur = conn.cursor()
    with open(file, 'r') as filename:
        reader = csv.reader(filename)
        next(reader)  # Skip the header row
        for row in reader:
            cur.execute('''
                INSERT INTO product(\
                    Product_ID, Order_Priority, Warehouse_Code, Ship_Mode\
       ,Delivery_Status, Item_Returned, Return_Reason, Payment_Type\
       ,Product_List_Title
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s)
            ''', row)
    conn.commit()
    cur.close()
    conn.close()

filepath = r'product.csv'
load_date_into_table(filepath)


In [ ]:
#load date into the date table
def load_date_into_table(file):
    conn = load_data_database()
    cur = conn.cursor()
    with open(file, 'r') as filename:
        reader = csv.reader(filename)
        next(reader)  # Skip the header row
        for row in reader:
            cur.execute('''
                INSERT INTO date( Date, Day, Month, Year
                ) VALUES (%s, %s, %s, %s)
            ''', row)
    conn.commit()
    cur.close()
    conn.close()

filepath = r'date.csv'
load_date_into_table(filepath)

In [ ]:
#load the transaction table
def load_date_into_table(file):
    conn = load_data_database()
    cur = conn.cursor()
    with open(file, 'r') as filename:
        reader = csv.reader(filename)
        next(reader)  # Skip the header row
        for row in reader:
            cur.execute('''
                INSERT INTO transaction (\
                Transaction_ID, Customer_ID, Product_ID, Date, Quantity\
       ,Unit_Price, Total_Cost, Discount_Rate, Sales_Channel\
       ,Taxable
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s,%s,%s)
            ''', row)
    conn.commit()
    cur.close()
    conn.close()

filepath = r'transaction.csv'
load_date_into_table(filepath)
print('date loaded successfull into {transaction} table')